Notebook version 1.0, 31 Aug 2021. Written by Joona Andersson / CSC - IT Center for Science Ltd. joona.andersson77@gmail.com

Licensed under the MIT license: http://www.opensource.org/licenses/mit-license.php

Tested on Kvasi, running QLM version 1.2.1: https://research.csc.fi/-/kvasi
***
# Performance of QAOA for MaxCut against classical exact and approximate algorithms
In this notebook, we compare the performance of QAOA for MaxCut without noise to the same circuit with imperfect gates and environmental noise. In addition, we compare these to a classical approximation algorithm, and to the exact value of the MaxCut calculated exhaustively. These simulations are computationally very expensive. **Simulations above 12 qubits take a very long time**. This makes simulating the effect of quantum noise on the algorithm challenging. Without a real quantum computer, one can still estimate the effects quantum noise will have on the algorithm at large problem sizes, however.

In [ ]:
import networkx as nx # this is a Python package for network/graph tools
import cvxpy as cvx # needed for the classical algorithm
import numpy as np
from qat.qpus import LinAlg
from qat.core.util import statistics
from qat.vsolve.qaoa import MaxCut
## Import tools for noisy simulation
from qat.quops import QuantumChannelKraus
from qat.hardware.default import HardwareModel
from qat.qpus import NoisyQProc
from qat.quops import ParametricPureDephasing, ParametricAmplitudeDamping
from qat.hardware import GatesSpecification
from itertools import product
import matplotlib.pyplot as plt
from qat.plugins import ObservableSplitter
from qat.plugins import ScipyMinimizePlugin
import matplotlib.pyplot as plt
import random

In the following code block, an exact classical algorithm is defined. It's been modified from code found in Google Code Archive (https://code.google.com/archive/p/maxcutpy/). The implementation details don't matter here, it's only used for comparison purposes.

In [ ]:
PARTITION = 'partition'
DEGREE = 'degree'

BLUE = 1
BLACK = -1

UNDECIDED = 0   # magenta
MARKED = 2      # red

# Return a dictionary representation of a cut
def partition_dictionary(G):
    return nx.get_node_attributes(G, PARTITION)

# Set node's blue class and black class
def set_partitions(G, blue_nodes, black_nodes):
    init_cut(G)
    cut(G, dict.fromkeys(blue_nodes, BLUE))
    cut(G, dict.fromkeys(black_nodes, BLACK))

# Return all partitions of a graph G as different sets
def get_partitions(G, nbunch=None):
    if nbunch is None:
        nbunch = G.nodes()

    blue_nodes = set()
    black_nodes = set()
    undecided_nodes = set()
    marked_nodes = set()

    for i in nbunch:
        if G.nodes[i][PARTITION] is BLUE:
            blue_nodes.add(i)

        elif G.nodes[i][PARTITION] is BLACK:
            black_nodes.add(i)

        elif G.nodes[i][PARTITION] is MARKED:
            marked_nodes.add(i)

        else:
            undecided_nodes.add(i)

    return blue_nodes, black_nodes, undecided_nodes, marked_nodes

# Return the number of edges between two sets of nodes
# Note: a and b should have no element in common.
def edges_between(G, a, b):
    return len(list(nx.edge_boundary(G, a, b)))

# Return the value of the cut.
def cut_edges(G, partition_dict=None):
    nbunch = None

    if partition_dict is not None:
        cut(G, partition_dict)

    blue_nodes, black_nodes = get_partitions(G, nbunch)[0:2]
    return edges_between(G, blue_nodes, black_nodes)

# Convert an integer to binary
def _integer_to_binary(i, n):
    rep = bin(i)[2:]
    return ('0' * (n - len(rep))) + rep

# Use a partition dictionary to cut a graph
def cut(G, partition_dict):
    nx.set_node_attributes(G, PARTITION, partition_dict)

# Cut a graph G using a binary operation
def binary_cut(G, int_cut, bin_cut=None):
    if bin_cut is None:
        bin_cut = _integer_to_binary(int_cut, G.number_of_nodes())

    for i, node in enumerate(G.nodes()):
        if bin_cut[i] is '0':
            G.nodes[node][PARTITION] = BLACK
        else:
            G.nodes[node][PARTITION] = BLUE

    return nx.get_node_attributes(G, PARTITION)

# Compute MaxCut by considereing all possible cuts
def brute_force_max_cut(G):
    max_cut_value = 0
    max_cut_ind = 0

    n = G.number_of_nodes()

    for i in range(1, 2 ** (n - 1)):
        cut_graph = nx.Graph(G)

        binary_cut(cut_graph, i)
        value = cut_edges(cut_graph)

        if value > max_cut_value:
            max_cut_value = value
            max_cut_ind = i

    binary_cut(G, max_cut_ind)
    return partition_dictionary(G), max_cut_value

In [ ]:
## Helper functions
def get_most_probable_state(result):
    samples = [sample for sample in result]
    samples.sort(key=lambda sample: sample.probability, reverse = True)
    return samples[0].state.value[0]

def maxcut_edges(state, graph):
    edges = 0
    for i, j in graph.edges():
        if state[i] != state[j]:
            edges += 1
    return edges
def compute_expectation(result, graph):
    avg = 0
    sum_count = 0
    for sample in result:
        obj = maxcut_edges(sample.state.value[0], graph)
        avg += obj
        sum_count += len(result) * sample.probability
    return avg/sum_count

def partition_to_bin(partition, dictionary = False):
    binary = ''
    if dictionary:
        for i in partition.values():
            if i == -1:
                binary += '0'
            else:
                binary += '1'
    else:
        for i in partition:
            if i == -1:
                binary += '0'
            else:
                binary += '1'
    return binary
       

One of the best classical approximate algorithms is the Goemans-Williamson Algorithm that achieves a 0.868 approximation ratio. This is a polynomial time algorithm.

In [ ]:
## A classical approximation alogorithm for MaxCut
def goemans_williamson(graph):
    """
    The Goemans-Williamson algorithm for solving the maxcut problem.
    Ref:
        Goemans, M.X. and Williamson, D.P., 1995. Improved approximation
        algorithms for maximum cut and satisfiability problems using
        semidefinite programming. Journal of the ACM (JACM), 42(6), 1115-1145
    Returns:
        np.ndarray: Graph coloring (+/-1 for each node)
        float:      The GW score for this cut.
        float:      The GW bound from the SDP relaxation
    """
    # Kudos: Original implementation by Nick Rubin, with refactoring and
    # cleanup by Jonathon Ward and Gavin E. Crooks
    laplacian = np.array(0.25 * nx.laplacian_matrix(graph).todense())

    # Setup and solve the GW semidefinite programming problem
    psd_mat = cvx.Variable(laplacian.shape, PSD=True)
    obj = cvx.Maximize(cvx.trace(laplacian @ psd_mat))
    constraints = [cvx.diag(psd_mat) == 1]  # unit norm
    prob = cvx.Problem(obj, constraints)
    prob.solve(solver=cvx.CVXOPT)

    evals, evects = np.linalg.eigh(psd_mat.value)
    sdp_vectors = evects.T[evals > float(1.0E-6)].T

    # Bound from the SDP relaxation
    bound = np.trace(laplacian @ psd_mat.value)

    random_vector = np.random.randn(sdp_vectors.shape[1])
    random_vector /= np.linalg.norm(random_vector)
    colors = np.sign([vec @ random_vector for vec in sdp_vectors])
    score = colors @ laplacian @ colors.T
    
    return colors, score, bound

In [ ]:
## Noise definitions
# X rotation gate
Xrot_fidelity = 0.995 # probability of success
Xrot_time = 30 # gate duration in nanoseconds

# H gate
H_fidelity = 0.99
H_time = 30

# CNOT gate
CNOT_fidelity = 0.98
CNOT_time = 200 

PH_fidelity = 0.99
PH_time = 30

## Environmetal noise parameters
T1 = 120000 # qubit's energy relaxation time in nanoseconds
T2 = 180000 # qubit's dephasing time in nanoseconds

px = Xrot_fidelity # probability that the RX(pi/2) rotation succeeds
noisy_RX = lambda theta : QuantumChannelKraus([np.sqrt(px)*np.array([[np.cos(theta/2), -np.sin(theta/2)*1j],
                                                     [-np.sin(theta/2)*1j, np.cos(theta/2)]]), # the RX(pi/2) gate -> gate succeeds
                                       np.sqrt(1-px)*np.identity(2)],  # the identity -> nothing happens
                                       name="noisy RX(pi/2)")           # name of the quantum operation/channel

# Noisy pi/2 radians Z rotation. Let's make this similar to the X rotation
ph = H_fidelity # probability that the RZ(pi/2) rotation succeeds
noisy_H = QuantumChannelKraus([np.sqrt(ph)*np.array([[1, 1],
                                                      [1, -1]]/np.sqrt(2)), # the RZ(pi/2) gate -> gate succeeds
                                      np.sqrt(1-ph)*np.identity(2)],        # the identity -> nothing happens
                                       name="noisy H")                      # name of the quantum operation/channel

pcnot = CNOT_fidelity # probability that the CNOT rotation succeeds
noisy_CNOT = QuantumChannelKraus([np.sqrt(pcnot)*np.array([[1, 0, 0, 0],
                                               [0, 1, 0, 0],
                                               [0, 0, 0, 1],
                                               [0, 0, 1, 0]]),         # the CNOT gate -> gate succeeds   
                                      np.sqrt(1-pcnot)*np.identity(4)],# the identity -> nothing happens
                                       name="noisy CNOT")

pph = PH_fidelity
noisy_PH = lambda theta : QuantumChannelKraus([np.sqrt(pph)*np.array([[1,0],
                                                                      [0, np.exp(theta*1j)]]),
                                              np.sqrt(1-pph)*np.identity(2)], name="noisy PH")

## Dictionary connecting the gate durations to the gate's name
gate_times = {"H": H_time,"RX": Xrot_time, "CNOT": CNOT_time, 'PH': PH_time} 

## Dictionary connecting the noisy quantum channels to gate names
quantum_channels = {"RX": noisy_RX,"H": noisy_H, "CNOT": noisy_CNOT, 'PH': noisy_PH}

## instanciate a GatesSpecification with gate times and the corresponding quantum channels
gates_spec = GatesSpecification(gate_times, quantum_channels)

## Amplitude Damping characterized by T_1
amp_damp = ParametricAmplitudeDamping(T_1 = T1)

## Pure Dephasing characterized by T_phi. The contribution of amplitude damping (T1) is removed 
## from transverse relaxation (T2) to give pure dephasing only (T_phi)
pure_deph = ParametricPureDephasing(T_phi = 1/(1/T2 - 1/(2*T1))) 

two_qbit_amp_damp = QuantumChannelKraus([np.kron(k1, k2) 
                                     for k1, k2 in product(amp_damp(tau=CNOT_time).kraus_operators,
                                                           amp_damp(tau=CNOT_time).kraus_operators)])

## Pure dephasing channel for two qubits. The logic is the same as above.
two_qbit_pure_deph = QuantumChannelKraus([np.kron(k1, k2)
                                     for k1, k2 in product(pure_deph(tau=CNOT_time).kraus_operators,
                                                           pure_deph(tau=CNOT_time).kraus_operators)])

gates_noise = {"RX" : lambda _: amp_damp(tau=Xrot_time)*pure_deph(tau=Xrot_time),  
               "H" : lambda : amp_damp(tau=H_time)*pure_deph(tau=H_time),
               "CNOT" : lambda : two_qbit_amp_damp*two_qbit_pure_deph,
               "PH" : lambda _: amp_damp(tau=PH_time)*pure_deph(tau=PH_time)}
quantum_channels = {"RX": noisy_RX,"H": noisy_H, "CNOT": noisy_CNOT, 'PH': noisy_PH}

gates_spec = GatesSpecification(gate_times, quantum_channels)

hw_model = HardwareModel(gates_spec, gate_noise=gates_noise, idle_noise=[amp_damp, pure_deph])


In [ ]:
nodes = 5
depth = 1
graph = nx.generators.random_graphs.erdos_renyi_graph(n=nodes, p = 0.5)
nx.draw(graph,with_labels=True) # vizualization of the randomly generated graph
problem = MaxCut(graph)
noisy_circuit = problem.qaoa_ansatz(depth=depth).circuit
H = problem.get_observable()

In [ ]:
optimizer = ScipyMinimizePlugin(method="COBYLA",
                            tol=1e-2,
                            options={"maxiter":150})

In [ ]:
## Funtion that performs the calculation with an ideal circuit
def get_ideal_circ_result():

    qpu = LinAlg()

    stack = optimizer | qpu
    
    job = noisy_circuit.to_job("OBS", observable=H, nbshots=0)
    
    result = stack.submit(job)
    
    params = eval(result.meta_data['parameters']) 
    gammas = params[0:depth]
    betas = params[depth:]
    
    circuit_optimal = noisy_circuit.bind_variables({key : var for key, var in zip(noisy_circuit.get_variables(),params)})

    job2 = circuit_optimal.to_job(nbshots=100000)
    result2 = qpu.submit(job2) 
    
    return result.value, result2

In [ ]:
## Run MaxCut Optimization step with imperfect gates and environmental noise
noisy_stack =  optimizer | ObservableSplitter() | NoisyQProc(hardware_model=hw_model)

noisy_job = noisy_circuit.to_job("OBS", observable=H, nbshots=0)

noisy_result = noisy_stack.submit(noisy_job)

## Compare the ideal result with the noisy result
energy, result = get_ideal_circ_result()

print(f'Final energy with the noisy circuit: {noisy_result.value}')
print(f'Final energy with the ideal circuit: {energy}')

In [ ]:
## Get the final answer
params = eval(noisy_result.meta_data['parameters']) 
gammas = params[:depth]
betas = params[depth:]

## Use the parameters in the circuit
circuit_optimal = noisy_circuit.bind_variables({key: var for key, 
                                                var in zip(noisy_circuit.get_variables(),params)})

noisy_job2 = circuit_optimal.to_job(nbshots=100000)

noisy_result2 = noisy_stack.submit(noisy_job2)

In [ ]:
## Compare the most probable result states from both circuits
noisy_state = get_most_probable_state(noisy_result2)
ideal_state = get_most_probable_state(result)
gw_partition, score, bound = goemans_williamson(graph)
optimal_partition, optimal_score = brute_force_max_cut(graph) 

print(f'Noisy circuit output state: {noisy_state}\n')
print(f'Ideal circuit output state: {ideal_state}\n')
print(f'Goemans_Williamson output partition: {partition_to_bin(gw_partition)}\n')
print(f'Optimal partition: {partition_to_bin(optimal_partition, dictionary=True)}\n')

print(f'Number of edges in the graph: {graph.number_of_edges()}')
print(f'Noisy MaxCut value: {maxcut_edges(noisy_state, graph)}')
print(f'Ideal MaxCut value: {maxcut_edges(ideal_state, graph)}')
print(f'Classical approximation result: {score}')
print(f'Optimal MaxCut value: {optimal_score}')